In [1]:
import pandas as pd
import ast

In [2]:
# pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
data = pd.read_csv('../data/raw/combined_df.csv')

In [4]:
data.head(5)

,name,url,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,longitude,latitude,summary,is_online_event,tickets_url,tags,event_id,data_source,image
0,Taller FAMILIAR de Lettering ¡DIBUJAMOS EL ABECEDARIO!,https://www.eventbrite.es/e/taller-familiar-de-lettering-dibujamos-el-abecedario-tickets-1312720154319,10:00,2025-05-10,12:00,2025-05-10,2:00:00,Tienda Cuadernos RUBIO,"9 Carrer de Sorní #bajo derecha, 46004 València",-0.368432,39.470320,"Ven y aprende a dibujar letras de forma creativa en familia en nuestro taller de lettering, ¡te esperamos en la tienda RUBIO!",False,https://www.eventbrite.com/checkout-external?eid=1312720154319,"['Crianza', 'Familia y educación', 'Clase, formación o taller', 'Taller', 'Lettering', 'Familiar', 'abecedario', 'dibujamos']",1312720154319,eventbrite,NaN
1,Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años,https://www.eventbrite.es/e/entradas-club-futuros-podcasters-taller-de-podcast-para-ninos-de-6-a-12-anos-1297237354849,17:30,2025-05-08,18:30,2025-05-08,1:00:00,"C. de Ramón y Cajal, 4","4 Calle de Ramón y Cajal, 28801 Alcalá de Henares",-3.365515,40.482542,Talleres de podcast mensuales para niños. Nos reunimos el primer jueves de cada mes. Ven a probar un taller o apúntate a todos hasta julio.,False,https://www.eventbrite.com/checkout-external?eid=1297237354849,"['Niños y jóvenes', 'Familia y educación', 'Clase, formación o taller', 'Radio', 'Audio', 'Taller', 'Podcast', 'Educacion', 'Comunicación', 'Curso', 'Ocio', 'Locutores', 'Microfono']",1297237354849,eventbrite,NaN
2,Jugar el arte en familia 3 DE MAYO,https://www.eventbrite.es/e/entradas-jugar-el-arte-en-familia-3-de-mayo-1302039748939,16:30,2025-05-03,17:30,2025-05-03,1:00:00,Casa del Lector,"14 Paseo de la Chopera, 28045 Madrid",-3.698443,40.392663,Explorando y creando juntos,False,https://www.eventbrite.com/checkout-external?eid=1302039748939,"['Niños y jóvenes', 'Familia y educación', 'Clase, formación o taller']",1302039748939,eventbrite,NaN
3,Certe MATHURIN dans AFFAIRES DE FAMILLE,https://www.eventbrite.fr/e/certe-mathurin-dans-affaires-de-famille-tickets-1058095154089,20:00,2025-05-23,21:15,2025-05-23,1:15:00,Tinta Roja Bar - Teatro,"17 Carrer de la Creu dels Molers, 08004 Barcelona",2.162020,41.374203,LE NOUVEAU SPECTACLE DE CERTE MATHURIN,False,https://www.eventbrite.com/checkout-external?eid=1058095154089,"['Crianza', 'Familia y educación', 'Concierto o actuación', 'Familia', 'Monólogo', 'Evento', 'Humor', 'Barcelona', 'Comedia en Barcelona', 'barcelona_comedy', 'mathurin', 'certe_mathurin', 'affaires_de_famille']",1058095154089,eventbrite,NaN
4,"Me Mimo Gestando, edición: ""Un nido consciente""",https://www.eventbrite.es/e/me-mimo-gestando-edicion-un-nido-consciente-tickets-1237774269159,10:00,2025-05-24,14:00,2025-05-24,4:00:00,ESPAI NERAIDA,"7 Camí del Transformador #B local A, 08394 Sant Vicenç de Montalt",2.509442,41.576094,"Solo con consciencia podremos abrirnos a la vida y cuidarla desde la salud y bienestar, favoreciendo el desplegar de toda la potencialidad.",False,https://www.eventbrite.com/checkout-external?eid=1237774269159,"['Crianza', 'Familia y educación', 'Conferencia', 'Consciência', 'Evento', 'Maternidad', 'me_mimo_gestando', 'un_nido_consciente']",1237774269159,eventbrite,NaN


In [5]:
df = data.copy()

In [6]:
import re
import pandas as pd

def extract_age_final(row):
    text = f"{row['name']} {row['summary']}"

    all_ranges_str = []
    min_ages = []
    max_ages = []

    # Buscar múltiples rangos en la misma línea (ej: 11 y 12 años)
    multi_range_patterns = re.findall(r'(\d+)\s*y\s*(\d+)\s*(?:years?|year?|anos?|año|años)\b', text, re.IGNORECASE)
    for min_age, max_age in multi_range_patterns:
        all_ranges_str.append(f"{min_age}-{max_age}")
        min_ages.append(int(min_age))
        max_ages.append(int(max_age))

    # Buscar rangos con guion o "a" (ej: 6-12 años)
    range_patterns = re.findall(r'(\d+)\s*(?:-|a)\s*(\d+)\s*(?:years?|year?|anos?|año|años)\b', text, re.IGNORECASE)
    for min_age, max_age in range_patterns:
        all_ranges_str.append(f"{min_age}-{max_age}")
        min_ages.append(int(min_age))
        max_ages.append(int(max_age))

    # Buscar patrones "A partir de"
    apartir_patterns = re.findall(r'A partir de\s*(\d+)\s*(?:years?|year?|anos?|año|años)?\b', text, re.IGNORECASE)
    for age in apartir_patterns:
        all_ranges_str.append(f"+{age}")
        min_ages.append(int(age))
        max_ages.append(14)  # Establecemos edad_max a 14

    # Buscar patrones con "+"
    plus_patterns = re.findall(r'\+(\d+)\s*(?:years?|year?|anos?|año|años)?\b', text, re.IGNORECASE)
    for age in plus_patterns:
        all_ranges_str.append(f"+{age}")
        min_ages.append(int(age))
        max_ages.append(14)  # Establecemos edad_max a 14

    # Buscar edades individuales
    single_patterns = re.findall(r'(\d+)\s*(?:years?|year?|anos?|año|años)\b', text, re.IGNORECASE)
    extracted_single = set()
    for age_range in all_ranges_str:
        if '-' in age_range:
            extracted_single.add(age_range.split('-')[0])
            extracted_single.add(age_range.split('-')[1])
        elif '+' in age_range:
            extracted_single.add(age_range[1:])
        else:
            extracted_single.add(age_range)

    for age_str in single_patterns:
        if age_str not in extracted_single:
            all_ranges_str.append(age_str)
            min_ages.append(int(age_str))
            max_ages.append(int(age_str))

    age_range_col = ' | '.join(all_ranges_str) if all_ranges_str else None

    final_min_age = min(min_ages) if min_ages else None
    final_max_age = max(max_ages) if max_ages else None

    return pd.Series({'age_range': age_range_col, 'min_age': final_min_age, 'max_age': final_max_age})

# Suponiendo que tu DataFrame se llama 'df'
df[['age_range', 'min_age', 'max_age']] = df.apply(extract_age_final, axis=1)

# Mostrar las filas relevantes con todas las columnas
print(df[df['age_range'].notna() | df['min_age'].notna() | df['max_age'].notna()][['name', 'summary', 'age_range', 'min_age', 'max_age']])

                                                                      name  \
1    Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años   
5                 Investigactiva. Especial Energía y movimiento 18 DE MAYO   
10                Investigactiva. Especial Energía y movimiento 1 DE JUNIO   
13                                         Cuánto Cuento!  (de 3 a 6 años)   
14                Cuánto Cuento!  (de 0 a 6 años) Especial Día de la Madre   
..                                                                     ...   
305                                       ¡Experimentos locos con el Agua!   
310                                            ¡Soy el chico/a del tiempo!   
311                                      ¿Hay alguien más aburrido que yo?   
312                                                ¿Jugamos con Europa...?   
313                                            ¿Qué hemos hecho en Madrid?   

                                                               

In [7]:
df['age_range'].value_counts().sum()

np.int64(135)

In [ ]:
# df.to_csv('../data/raw/combined_df_with_age_range.csv', index=False)

In [9]:
filtrado = df[df['age_range'].notna()][['name','summary','age_range','min_age','max_age']]

In [10]:
filtrado.head(20)

,name,summary,age_range,min_age,max_age
1,Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años,Talleres de podcast mensuales para niños. Nos reunimos el primer jueves de cada mes. Ven a probar un taller o apúntate a todos hasta julio.,6-12,6.0,12.0
5,Investigactiva. Especial Energía y movimiento 18 DE MAYO,Espacio de experimentación científica para niñas y niños de 2 a 8 años,2-8,2.0,8.0
10,Investigactiva. Especial Energía y movimiento 1 DE JUNIO,Espacio de experimentación científica para niñas y niños de 2 a 8 años,2-8,2.0,8.0
13,Cuánto Cuento! (de 3 a 6 años),¡Te esperamos en nuestros cuentos dramatizados!,3-6,3.0,6.0
14,Cuánto Cuento! (de 0 a 6 años) Especial Día de la Madre,"Ven y disfruta de una mañana llena de historias, risas y diversión con nuestros cuenta cuentos más especiales.",0-6,0.0,6.0
17,BAJO EL AGUA_Tolon tell on (MENUTSBARRIS) Teatro.,Obra de teatro sobre el cuidado del mar. +3 años.,+3,3.0,14.0
18,EL REMENDÓN DE LA MUERTE (MENUTSBARRIS) Títelles (+ 6 años).,El agua de la vida sirve para curar cualquier enfermedad.,6,6.0,6.0
20,A la mar salada,"Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. Recuerda que las niñas y niños deberán ir acompañados por una persona adulta. Para asistir es necesario realizar inscripción previa a través de este enlace . Te esperamos en la biblioteca para pasar un buen rato lleno de fantasía, de la mano de Margalida Alberti.",+2,2.0,14.0
21,A taste of the moon,"Vamos a leer en inglés y a divertirnos en la biblioteca. Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. La entrada es libre hasta completar aforo. Recuerda que los niños y niñas deberán ir acompañados por una persona adulta. Te esperamos en la biblioteca para pasar un buen rato lleno de fantasía, de la mano de...",+2,2.0,14.0
22,A taste of the moon,"Vamos a leer en inglés y a divertirnos en la biblioteca. Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. La entrada es libre hasta completar aforo. Recuerda que los niños y niñas deberán ir acompañados por una persona adulta. Te esperamos en la biblioteca para pasar un buen rato lleno de fantasía, de la mano de M...",+2,2.0,14.0


In [ ]:
# Asegúrese de que las 'etiquetas' sean listas adecuadas (no cadenas)
def safe_parse_list(tag_str):
    try:
        return ast.literal_eval(tag_str) if isinstance(tag_str, str) else tag_str
    except:
        return []

df['tags'] = df['tags'].apply(safe_parse_list)

# Filtrar filas donde data_source == 'eventbrite'
eventbrite_df = df[df['data_source'] == 'eventbrite']

# Aplanar la lista de etiquetas en todas las filas
all_tags = []
for tag_list in eventbrite_df['tags']:
    all_tags.extend([tag.strip() for tag in tag_list])  # Tira para limpiar espacios en blanco

# Obtener tags unicas
unique_tags = sorted(set(all_tags))

In [ ]:
english_tags = [
    'Audio', 'Baby', 'Babygroup', 'Bookshop', 'Children', 'Children & Youth',
    'Childrenandfamilies', 'Childrens', 'Class, Training, or Workshop', 'Community',
    'Concert or Performance', 'Conference', 'Event', 'Family', 'Family & Education',
    'Familyworkshop', 'Gathering', 'Humour', 'Kids', 'Lettering', 'Parenting',
    'Podcast', 'Radio', 'Standup', 'Standupcomedy', 'Storyteller', 'Storytelling',
    'Workshop', 'kids_events'
]

spanish_tags = [
    'Audio', 'Bebé', 'Grupo de bebés', 'Librería', 'Niños', 'Niños y jóvenes',
    'Niños y familias', 'Niños', 'Clase, formación o taller', 'Comunidad',
    'Concierto o actuación', 'Conferencia', 'Evento', 'Familia', 'Familia y educación',
    'Taller familiar', 'Reunión', 'Humor', 'Niños', 'Lettering', 'Crianza',
    'Podcast', 'Radio', 'Monólogo', 'Comedia en Barcelona', 'Narrador',
    'Narración de cuentos', 'Taller', 'eventos_infantiles'
]

translation_dict = dict(zip(english_tags, spanish_tags))

import ast

# Asegurarse de que 'etiquetas' sea una lista
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Función para traducir listas de etiquetas individuales
def translate_tag_list(tag_list):
    return [translation_dict.get(tag.strip(), tag.strip()) for tag in tag_list]

# Aplicar solo a filas donde data_source == 'eventbrite'
df.loc[df['data_source'] == 'eventbrite', 'tags'] = (
    df.loc[df['data_source'] == 'eventbrite', 'tags'].apply(translate_tag_list)
)


In [ ]:
# df.to_csv('../data/raw/combined_df_age_range_tags_spa.csv', index=False)